## Práctica examen 2019-2020

__ETL, MDS__

__Valentina Díaz Torres__

##  The World Bank's international debt data

<p>No es que los humanos solo tengamos deudas para administrar nuestras necesidades. Un país también puede endeudarse para administrar su economía. Por ejemplo, el gasto en infraestructura es un ingrediente costoso requerido para que los ciudadanos de un país lleven una vida cómoda. El Banco Mundial es la organización que proporciona deuda a los países.</p>

<!-- <p>En este notebook, vamos a analizar los datos de la deuda internacional recopilados por el Banco Mundial. El conjunto de datos contiene información sobre el monto de la deuda (en USD) que deben los países en desarrollo en varias categorías.</p>  -->
    

"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD
"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD
"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD
"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD
"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD
"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD
"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD
"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD
"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD
"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD
"PPG, official creditors (INT, current US$)",DT.INT.OFFT.CD
"Principal repayments on external debt, long-term (AMT, current US$)",DT.AMT.DLXF.CD
"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",DT.INT.DPNG.CD
"PPG, bonds (AMT, current US$)",DT.AMT.PBND.CD
"PPG, bonds (INT, current US$)",DT.INT.PBND.CD
"PPG, commercial banks (AMT, current US$)",DT.AMT.PCBK.CD
"PPG, commercial banks (DIS, current US$)",DT.DIS.PCBK.CD
"PPG, commercial banks (INT, current US$)",DT.INT.PCBK.CD
"PPG, other private creditors (AMT, current US$)",DT.AMT.PROP.CD
"PPG, other private creditors (DIS, current US$)",DT.DIS.PROP.CD
"PPG, other private creditors (INT, current US$)",DT.INT.PROP.CD
"PPG, private creditors (AMT, current US$)",DT.AMT.PRVT.CD
"PPG, private creditors (DIS, current US$)",DT.DIS.PRVT.CD
"PPG, private creditors (INT, current US$)",DT.INT.PRVT.CD
"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",DT.AMT.DPNG.CD

<p>Vamos a encontrar las respuestas a preguntas como:

<p>¿Cuál es el monto total de la deuda que deben los países enumerados en el conjunto de datos?
<p>¿Qué país posee la cantidad máxima de deuda y cómo se ve esa cantidad?
<p>¿Cuál es el monto promedio de la deuda de los países a través de diferentes indicadores de deuda?
    
Además tenemos otro dataset en el que encontramos información histórica de algunos índices de desarrollo, entre los que se encuentran algunos de deuda como son:

Series Name,Series Code
"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN

"Central government debt, total (current LCU)",GC.DOD.TOTL.CN

"Central government debt, total (% of GDP)",GC.DOD.TOTL.GD.ZS

__PREGUNTAS Y SOLUCIONES__

#### 1. Inicializar y cargar el contexto spark

In [1]:
# Librerias e inicializacion y carga del contexto spark

import re
from pyspark.sql import functions as f
from pyspark.sql import Row
import numpy as np
import pandas as pd
from pyspark import SparkContext
pd.options.display.max_columns = None 
pd.options.display.max_rows = None 
from pyspark.sql import SQLContext
sc= SparkContext()
sqlContext = SQLContext(sc)
from pyspark.sql import *
from pyspark.sql import SQLContext, functions as F, Row
from pyspark.sql.types import *
from pyspark.sql.types import StringType, IntegerType
import findspark
findspark.init()
import pyspark # Call this only after findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
from pyspark.sql.functions import desc
from pyspark.sql.functions import col


__Carga de datos__

In [2]:
# Carga de las dos bases de datos

# Desarrollo

i_desarrollo = spark.read.format('csv').option('header',True).option('multiLine', True).load('indicadores_desarrollo.csv')

# Deuda

i_deuda = spark.read.format('csv').option('header',True).option('multiLine', True).load('indicadores_deuda.csv')             

__Explorar el aspecto que tienen los datos__

In [3]:
# i_desarrollo tiene 134 filas

i_desarrollo.count()

134

In [4]:
#i_deuda tiene 124 filas

i_deuda.count()

124

In [5]:
#columnas existentes en i_desarrollo
i_desarrollo.columns

['country_name',
 'country_code',
 'anho',
 'GC.DOD.TOTL.CN',
 'GC.DOD.TOTL.GD.ZS',
 'SP.DYN.CBRT.IN']

In [6]:
#columnas existentes en i_deuda

i_deuda.columns

['country_name',
 'country_code',
 'DT.AMT.BLAT.CD',
 'DT.AMT.DLXF.CD',
 'DT.AMT.DPNG.CD',
 'DT.AMT.MLAT.CD',
 'DT.AMT.OFFT.CD',
 'DT.AMT.PBND.CD',
 'DT.AMT.PCBK.CD',
 'DT.AMT.PROP.CD',
 'DT.AMT.PRVT.CD',
 'DT.DIS.BLAT.CD',
 'DT.DIS.DLXF.CD',
 'DT.DIS.MLAT.CD',
 'DT.DIS.OFFT.CD',
 'DT.DIS.PCBK.CD',
 'DT.DIS.PROP.CD',
 'DT.DIS.PRVT.CD',
 'DT.INT.BLAT.CD',
 'DT.INT.DLXF.CD',
 'DT.INT.DPNG.CD',
 'DT.INT.MLAT.CD',
 'DT.INT.OFFT.CD',
 'DT.INT.PBND.CD',
 'DT.INT.PCBK.CD',
 'DT.INT.PROP.CD',
 'DT.INT.PRVT.CD']

In [7]:
#aspecto de i_desarrollo
i_desarrollo.take(5)

[Row(country_name='Albania', country_code='ALB', anho='2014 [YR2014]', GC.DOD.TOTL.CN='1023040800000', GC.DOD.TOTL.GD.ZS='73.3202469198429', SP.DYN.CBRT.IN='12.259'),
 Row(country_name='Albania', country_code='ALB', anho='2015 [YR2015]', GC.DOD.TOTL.CN='1145500940000', GC.DOD.TOTL.GD.ZS='79.8644448747259', SP.DYN.CBRT.IN='12.197'),
 Row(country_name='Albania', country_code='ALB', anho='2016 [YR2016]', GC.DOD.TOTL.CN='1188814490000', GC.DOD.TOTL.GD.ZS='80.7355748396923', SP.DYN.CBRT.IN='12.08'),
 Row(country_name='Australia', country_code='AUS', anho='2014 [YR2014]', GC.DOD.TOTL.CN='674700000', GC.DOD.TOTL.GD.ZS='0.042207528166503', SP.DYN.CBRT.IN='13.2'),
 Row(country_name='Australia', country_code='AUS', anho='2015 [YR2015]', GC.DOD.TOTL.CN='762718000', GC.DOD.TOTL.GD.ZS='0.0469480198522591', SP.DYN.CBRT.IN='12.9')]

In [8]:
#aspecto de i_deuda
i_deuda.take(3)

[Row(country_name='Afghanistan', country_code='AFG', DT.AMT.BLAT.CD='61739336.9', DT.AMT.DLXF.CD='100847181.9', DT.AMT.DPNG.CD=None, DT.AMT.MLAT.CD='39107845', DT.AMT.OFFT.CD='100847181.9', DT.AMT.PBND.CD=None, DT.AMT.PCBK.CD=None, DT.AMT.PROP.CD=None, DT.AMT.PRVT.CD=None, DT.DIS.BLAT.CD='49114729.4', DT.DIS.DLXF.CD='72894453.7', DT.DIS.MLAT.CD='23779724.3', DT.DIS.OFFT.CD='72894453.7', DT.DIS.PCBK.CD=None, DT.DIS.PROP.CD=None, DT.DIS.PRVT.CD=None, DT.INT.BLAT.CD='39903620.1', DT.INT.DLXF.CD='53239440.1', DT.INT.DPNG.CD=None, DT.INT.MLAT.CD='13335820', DT.INT.OFFT.CD='53239440.1', DT.INT.PBND.CD=None, DT.INT.PCBK.CD=None, DT.INT.PROP.CD=None, DT.INT.PRVT.CD=None),
 Row(country_name='Albania', country_code='ALB', DT.AMT.BLAT.CD='54250280.6', DT.AMT.DLXF.CD='790248675.2', DT.AMT.DPNG.CD='514185620', DT.AMT.MLAT.CD='182197616.7', DT.AMT.OFFT.CD='236447897.3', DT.AMT.PBND.CD='0', DT.AMT.PCBK.CD='39445139.5', DT.AMT.PROP.CD='170018.4', DT.AMT.PRVT.CD='39615157.9', DT.DIS.BLAT.CD='76050616.1

In [9]:
#renombrar algunas de las columnas de i_desarrollo que más tarde utilizaré, para poder trabajar con ellas,
#ya que contienen puntos.

#DEUDA

i_deuda = i_deuda.withColumnRenamed('DT.AMT.BLAT.CD','dt_amt_blat_cd')
i_deuda = i_deuda.withColumnRenamed('DT.AMT.MLAT.CD' , 'dt_amt_mlat_cd')
i_deuda = i_deuda.withColumnRenamed('DT.DIS.BLAT.CD' , 'dt_dis_blat_cd')
i_deuda = i_deuda.withColumnRenamed('DT.INT.BLAT.CD' , 'dt_int_blat_cd')
i_deuda = i_deuda.withColumnRenamed('DT.AMT.BLAT.CD' , 'dt_amt_blat_cd')
i_deuda = i_deuda.withColumnRenamed('SP.DYN.CBRT.IN' , 'sp_dyn_crbt_in')
i_deuda = i_deuda.withColumnRenamed('DT.AMT.DLXF.CD' , 'dt_amt_dlxf_cd')

#DESARROLLO


i_desarrollo = i_desarrollo.withColumnRenamed('anho' , 'year')
i_desarrollo = i_desarrollo.withColumnRenamed('GC.DOD.TOTL.CN' , 'gc_dod_tolt_cn')
i_desarrollo = i_desarrollo.withColumnRenamed('GC.DOD.TOTL.GD.ZS' , 'gc_dod_totl_gd_zs')
i_desarrollo = i_desarrollo.withColumnRenamed('SP.DYN.CBRT.IN' , 'sp_dyn_cbrt_in')


##### 2. Número de países distintos en cada dataset. Coinciden?

El número de países que aparecen en cada dataset es el siguiente:

In [10]:
paises_deuda = i_deuda.groupBy("country_name").sum().distinct().collect()
len(paises_deuda)

124

In [11]:
paises_desarrollo = i_desarrollo.groupBy("country_name").sum().distinct().collect()
len(paises_desarrollo)

51

El número de países existentes en desarrollo y en deuda es diferente. Anteriormente, se han mostrado los países que aparecen en una y otra base de datos. El total de países de deuda es de 124 y en desarrollo 51.

A continuación se muestran todos los países que contienen los dos datasets, con el número de registros que cada uno contine, es decir el número de veces que el país aparece.

In [12]:
i_deuda.groupBy("country_name").count().show(124)

+--------------------+-----+
|        country_name|count|
+--------------------+-----+
|          South Asia|    1|
|                Chad|    1|
|            Paraguay|    1|
|    Congo, Dem. Rep.|    1|
|             Senegal|    1|
|          Cabo Verde|    1|
|Least developed c...|    1|
|      Macedonia, FYR|    1|
|              Guyana|    1|
|             Eritrea|    1|
|         Philippines|    1|
|            Djibouti|    1|
|               Tonga|    1|
|                Fiji|    1|
|              Turkey|    1|
|              Malawi|    1|
|             Comoros|    1|
|         Afghanistan|    1|
|            Cambodia|    1|
|              Jordan|    1|
|            Maldives|    1|
|              Rwanda|    1|
|               Sudan|    1|
|              Kosovo|    1|
|           Sri Lanka|    1|
|            Dominica|    1|
|             Algeria|    1|
|                Togo|    1|
|              Angola|    1|
|  Iran, Islamic Rep.|    1|
|             Ecuador|    1|
|             

In [13]:
i_desarrollo.groupBy("country_name").count().show(51)

+--------------------+-----+
|        country_name|count|
+--------------------+-----+
|         Philippines|    1|
|               Tonga|    3|
|            Malaysia|    3|
|           Singapore|    3|
|              Turkey|    3|
|              Malawi|    4|
|              Jordan|    2|
|               Palau|    3|
|           Sri Lanka|    2|
|          San Marino|    2|
|             Albania|    3|
|                Peru|    3|
|           St. Lucia|    1|
|       United States|    3|
|             Belarus|    3|
| St. Kitts and Nevis|    1|
|               Spain|    3|
|            Barbados|    2|
|  Russian Federation|    3|
|             Ireland|    3|
|            Thailand|    2|
|              Bhutan|    3|
|             Ukraine|    3|
|             Uruguay|    3|
|             Georgia|    3|
|           Indonesia|    3|
|           Guatemala|    3|
|         Korea, Rep.|    3|
|             Namibia|    1|
|         Switzerland|    3|
|              Zambia|    2|
|             

##### 3. Total de deuda contraida por cada pais: total amount of debt (in USD) DT.AMT.MLAT.CD

En la siguiente tabla, se han calculado los totales de duda y agrupados por cada país, de tal forma que quede todo recogido.

In [14]:
deudaPais = i_deuda.groupby(["country_name"]).agg(f.sum('dt_amt_mlat_cd').alias("deudaTotal_pais"))
deudaPais.show()

+--------------------+---------------+
|        country_name|deudaTotal_pais|
+--------------------+---------------+
|          South Asia| 7.8517399295E9|
|                Chad|   9.57387095E7|
|            Paraguay|  2.076620309E8|
|    Congo, Dem. Rep.|   1.43105462E8|
|             Senegal|  1.704379291E8|
|          Cabo Verde|   3.99701249E7|
|Least developed c...| 4.0842755242E9|
|      Macedonia, FYR|  1.859528672E8|
|              Guyana|   3.46518065E7|
|             Eritrea|   1.54824754E7|
|         Philippines|  7.390564273E8|
|            Djibouti|   3.42549092E7|
|               Tonga|      2791253.2|
|                Fiji|      9082442.6|
|              Turkey|  2.932698678E9|
|              Malawi|   3.56598314E7|
|             Comoros|      2182053.1|
|         Afghanistan|    3.9107845E7|
|            Cambodia|   7.23070018E7|
|              Jordan|  2.704987155E8|
+--------------------+---------------+
only showing top 20 rows



##### 4. Media de los indicadores de deuda (tabla uno): DT.AMT.BLAT.CD, DT.DIS.BLAT.CD, DT.INT.BLAT.CD

* DT.AMT.BLAT.CD

In [15]:
# Media por pais según el indicador DT.AMT.BLAT.CD

media_1 = i_deuda.groupBy("country_name").agg(f.avg("dt_amt_blat_cd").alias("media_1_pais")).show()

+--------------------+----------------+
|        country_name|    media_1_pais|
+--------------------+----------------+
|          South Asia|  6.4378044481E9|
|                Chad|    8.23572149E7|
|            Paraguay|    5.83141627E7|
|    Congo, Dem. Rep.|   1.164523294E8|
|             Senegal|   1.322410302E8|
|          Cabo Verde|    1.17676543E7|
|Least developed c...|1.29447030366E10|
|      Macedonia, FYR|    7.05705503E7|
|              Guyana|    2.17585257E7|
|             Eritrea|    1.56276024E7|
|         Philippines|   6.704754179E8|
|            Djibouti|    4.88032791E7|
|               Tonga|       7554054.0|
|                Fiji|    3.73828766E7|
|              Turkey|   6.464638798E8|
|              Malawi|    2.17625417E7|
|             Comoros|       2610216.0|
|         Afghanistan|    6.17393369E7|
|            Cambodia|   2.233002736E8|
|              Jordan|   2.302044672E8|
+--------------------+----------------+
only showing top 20 rows



In [16]:
# Media total

i_deuda.agg({"dt_amt_blat_cd": "avg"}).show()

+-------------------+
|avg(dt_amt_blat_cd)|
+-------------------+
|7.126196346598358E8|
+-------------------+



* DT.DIS.BLAT.CD

In [17]:
# Media por pais según DT.DIS.BLAT.CD

media_2 = i_deuda.groupBy("country_name").agg(f.avg("dt_dis_blat_cd").alias("media_2_pais")).show()

+--------------------+----------------+
|        country_name|    media_2_pais|
+--------------------+----------------+
|          South Asia|1.72691140409E10|
|                Chad|    2.79876609E7|
|            Paraguay|   1.324315404E8|
|    Congo, Dem. Rep.|    1.03263632E8|
|             Senegal|   5.486485091E8|
|          Cabo Verde|    3.71784448E7|
|Least developed c...|2.33339563884E10|
|      Macedonia, FYR|   1.013965784E8|
|              Guyana|    3.55846841E7|
|             Eritrea|       2678802.6|
|         Philippines|   5.441485278E8|
|            Djibouti|    8.56870121E7|
|               Tonga|            null|
|                Fiji|       3902886.7|
|              Turkey|   4.152690754E8|
|              Malawi|    2.65599433E7|
|             Comoros|       8492726.4|
|         Afghanistan|    4.91147294E7|
|            Cambodia|   4.766838294E8|
|              Jordan|   1.754843073E8|
+--------------------+----------------+
only showing top 20 rows



In [18]:
# Media total del indicador

i_deuda.agg({"dt_dis_blat_cd": "avg"}).show()

+-------------------+
|avg(dt_dis_blat_cd)|
+-------------------+
| 1.22313929039823E9|
+-------------------+



* DT.INT.BLAT.CD

In [19]:
# Media por país del indicador DT.INT.BLAT.CD

media_3 = i_deuda.groupBy("country_name").agg(f.avg("dt_int_blat_cd").alias("media_2_pais")).show()

+--------------------+--------------+
|        country_name|  media_2_pais|
+--------------------+--------------+
|          South Asia|2.0049174493E9|
|                Chad|  1.18827709E7|
|            Paraguay|     5158922.3|
|    Congo, Dem. Rep.|  6.22606531E7|
|             Senegal|  6.28565493E7|
|          Cabo Verde|     4216236.3|
|Least developed c...|3.0298263523E9|
|      Macedonia, FYR|  1.41276186E7|
|              Guyana|     4885618.3|
|             Eritrea|     1556112.1|
|         Philippines| 1.042379929E8|
|            Djibouti|  4.48788819E7|
|               Tonga|     1559371.0|
|                Fiji|     6175526.5|
|              Turkey| 1.104107777E8|
|              Malawi|     7850229.4|
|             Comoros|      782831.5|
|         Afghanistan|  3.99036201E7|
|            Cambodia|  6.38366832E7|
|              Jordan|  6.21361037E7|
+--------------------+--------------+
only showing top 20 rows



In [20]:
# Deuda total indicador

i_deuda.agg({"dt_int_blat_cd": "avg"}).show()

+-------------------+
|avg(dt_int_blat_cd)|
+-------------------+
|1.640932856090164E8|
+-------------------+



##### 5. Los 20 paises con DT.AMT.DLXF.CD más alto

Para obtener los 20 países con mayor indicador DT.AMT.DLXF.CD, en primer lugar, se agrupa por país y por la columna del indicador. Una vez rescatados los datos, se ordenan de mayor a menor, para quedarse con los 20 primeros y así obtener el "top 20" del indicador.

In [21]:
# Agrupar por país y por la columna del indicador

top_20 = i_deuda.groupBy("country_name").agg(f.sum("dt_amt_dlxf_cd").alias("top_20"))


In [22]:
# Ordenar de mayor a menor y mostrar solo 20

top_20.sort(f.col("top_20").desc()).show(20)

+--------------------+----------------+
|        country_name|          top_20|
+--------------------+----------------+
|               China|9.62186208357E10|
|              Brazil|9.00418403041E10|
|  Russian Federation|6.65897618335E10|
|              Turkey|5.15550310058E10|
|          South Asia|4.87562958982E10|
|               India|3.19235070008E10|
|           Indonesia|3.09161126538E10|
|          Kazakhstan|2.74820936864E10|
|              Mexico| 2.5218503927E10|
|Least developed c...|2.51970292994E10|
|            IDA only| 2.0483289208E10|
|             Romania|1.40137833504E10|
|            Colombia|1.19856744387E10|
|              Angola|1.10670456281E10|
|            Cameroon|1.04048149602E10|
|       Venezuela, RB|  9.8786592072E9|
|    Egypt, Arab Rep.|  9.6921141769E9|
|             Lebanon|  9.5069196696E9|
|        South Africa|  9.4742575519E9|
|             Vietnam|  8.8735059092E9|
+--------------------+----------------+
only showing top 20 rows



##### 6. Pais con los datos informados todos los años.

Para conocer qué país del dataset de desarrollo ha proporcionado datos todos los años, en primer lugar, se realiza la función count, para saber la frecuencia de cada país. Se observa, que el valor máximo es 4, 4 veces, que coincide con el número de años que aparecen en el dataset. A continuación se realiza un filtrado de aquellos que cumplan con la condición de aparecer 4 veces en los datos.


In [23]:
i_desarrollo.groupBy("country_name").count().show()

+-------------------+-----+
|       country_name|count|
+-------------------+-----+
|        Philippines|    1|
|              Tonga|    3|
|           Malaysia|    3|
|          Singapore|    3|
|             Turkey|    3|
|             Malawi|    4|
|             Jordan|    2|
|              Palau|    3|
|          Sri Lanka|    2|
|         San Marino|    2|
|            Albania|    3|
|               Peru|    3|
|          St. Lucia|    1|
|      United States|    3|
|            Belarus|    3|
|St. Kitts and Nevis|    1|
|              Spain|    3|
|           Barbados|    2|
| Russian Federation|    3|
|            Ireland|    3|
+-------------------+-----+
only showing top 20 rows



Se comprueba que los países con un número 4 en la columna de la tabla anterior "count" son los que tienen datos informados todos los años. Para comprobar qué país o países son estos, se realiza un filter.

In [24]:
totales_paises = i_desarrollo.groupBy("country_name").count().filter("count == 4")
totales_paises.show()

+------------+-----+
|country_name|count|
+------------+-----+
|      Malawi|    4|
+------------+-----+



Finalmente, el resultado es Malawi.

En el caso del dataset de deuda, no tiene sentido el estudio, ya que todos los países se repiten una vez solo.

In [25]:
i_deuda.groupBy("country_name").count().show()

+--------------------+-----+
|        country_name|count|
+--------------------+-----+
|          South Asia|    1|
|                Chad|    1|
|            Paraguay|    1|
|    Congo, Dem. Rep.|    1|
|             Senegal|    1|
|          Cabo Verde|    1|
|Least developed c...|    1|
|      Macedonia, FYR|    1|
|              Guyana|    1|
|             Eritrea|    1|
|         Philippines|    1|
|            Djibouti|    1|
|               Tonga|    1|
|                Fiji|    1|
|              Turkey|    1|
|              Malawi|    1|
|             Comoros|    1|
|         Afghanistan|    1|
|            Cambodia|    1|
|              Jordan|    1|
+--------------------+-----+
only showing top 20 rows



##### 7. Media anual de los distintos indicadores de desarrollo

En este apartado se realizan las medias anuales de los tres indicadores de desarrollo que aparecen en el dataset. Para ello, se puede observar el nombre de las columnas del mismo.

In [26]:
i_desarrollo.columns

['country_name',
 'country_code',
 'year',
 'gc_dod_tolt_cn',
 'gc_dod_totl_gd_zs',
 'sp_dyn_cbrt_in']

Una vez que estos se conocen, se agrupa por año y se realiza la media del indicador, de tal forma que todos los datos quedan recogidos en una tabla.

In [31]:
media_1 = i_desarrollo.select("gc_dod_tolt_cn","year").groupBy("year").agg(f.avg("gc_dod_tolt_cn").alias("media_1"))
media_1.show()

+-------------+--------------------+
|         year|             media_1|
+-------------+--------------------+
|2016 [YR2016]|1.588138215480659...|
|2014 [YR2014]|1.034427587329532...|
|2017 [YR2017]| 2.23985861996269E12|
|2015 [YR2015]|1.318244039159024...|
+-------------+--------------------+



In [32]:

media_2 = i_desarrollo.select("gc_dod_totl_gd_zs","year").groupBy("year").agg(f.avg("gc_dod_totl_gd_zs").alias("media_2"))
media_2.show()

+-------------+------------------+
|         year|           media_2|
+-------------+------------------+
|2016 [YR2016]|58.796949624008356|
|2014 [YR2014]|56.843914344384395|
|2017 [YR2017]|   48.659564238322|
|2015 [YR2015]| 58.36559834488036|
+-------------+------------------+



In [35]:
media_3 = i_desarrollo.select("sp_dyn_cbrt_in","year").groupBy("year").agg(f.avg("sp_dyn_cbrt_in").alias("media_3"))
media_3.show()

+-------------+------------------+
|         year|           media_3|
+-------------+------------------+
|2016 [YR2016]| 16.14433333333333|
|2014 [YR2014]|          17.58262|
|2017 [YR2017]|            34.593|
|2015 [YR2015]|16.798477272727276|
+-------------+------------------+



##### 8. Podrías decirme el total de deuda acumulada DT.AMT.MLAT.CD por los 10 países con un valor en media menor de SP.DYN.CBRT.IN

Para esta pregunta, se realizan en primer lugar los totales de tasa de natalidad de todos los países del dataset. Luego, se hace lo mismo con los totales del indicador de deuda de DT.AMT.MLAT.CD. Una vez que tengo estos datos, se ordenan con el fin de que aparezcan los menores arriba, ya que se están buscando los 10 países con __menor__ valor en media de tasa de natalidad. 

In [36]:
media_paises_desarrollo = i_desarrollo.groupBy("country_name").agg(f.avg('sp_dyn_cbrt_in').alias("media_desarrollo"))
media_paises_desarrollo.show()

+-------------------+------------------+
|       country_name|  media_desarrollo|
+-------------------+------------------+
|        Philippines|            22.935|
|              Tonga|25.362333333333336|
|           Malaysia|17.017333333333333|
|          Singapore| 9.633333333333333|
|             Turkey|16.717333333333332|
|             Malawi|            35.556|
|             Jordan|            24.305|
|              Palau|13.133333333333333|
|          Sri Lanka|            16.729|
|         San Marino|              8.05|
|            Albania|12.178666666666667|
|               Peru|18.564666666666668|
|          St. Lucia|            12.502|
|      United States|12.366666666666665|
|            Belarus|12.466666666666667|
|St. Kitts and Nevis|              12.6|
|              Spain|               9.0|
|           Barbados|            10.861|
| Russian Federation|13.166666666666666|
|            Ireland|              13.9|
+-------------------+------------------+
only showing top

In [43]:
total_deuda_porPaises = i_deuda.groupBy("country_name").agg(f.sum('dt_amt_mlat_cd').alias('total_deuda'))
total_deuda_porPaises.show()

+--------------------+--------------+
|        country_name|   total_deuda|
+--------------------+--------------+
|          South Asia|7.8517399295E9|
|                Chad|  9.57387095E7|
|            Paraguay| 2.076620309E8|
|    Congo, Dem. Rep.|  1.43105462E8|
|             Senegal| 1.704379291E8|
|          Cabo Verde|  3.99701249E7|
|Least developed c...|4.0842755242E9|
|      Macedonia, FYR| 1.859528672E8|
|              Guyana|  3.46518065E7|
|             Eritrea|  1.54824754E7|
|         Philippines| 7.390564273E8|
|            Djibouti|  3.42549092E7|
|               Tonga|     2791253.2|
|                Fiji|     9082442.6|
|              Turkey| 2.932698678E9|
|              Malawi|  3.56598314E7|
|             Comoros|     2182053.1|
|         Afghanistan|   3.9107845E7|
|            Cambodia|  7.23070018E7|
|              Jordan| 2.704987155E8|
+--------------------+--------------+
only showing top 20 rows



In [47]:
union1 = total_deuda_porPaises.join(media_paises_desarrollo, on="country_name", how = "inner")

Estos son los 10 países con menor media, ordenadas de menor a mayor, para poder comparar y su total de deuda.

In [46]:
ordenar = union1.orderBy("media_desarrollo", ascending = True)
ordenar.show(10)

+--------------------+--------------+------------------+
|        country_name|   total_deuda|  media_desarrollo|
+--------------------+--------------+------------------+
|             Ukraine|1.0724728303E9|              10.6|
|             Moldova| 1.358428655E8|10.604333333333335|
|            Thailand|  3.90715715E7|           10.9175|
|             Albania| 1.821976167E8|12.178666666666667|
|             Belarus|  5.95438826E8|12.466666666666667|
|           St. Lucia|  1.50192163E7|            12.502|
|  Russian Federation|    1.416695E8|13.166666666666666|
|             Georgia| 1.440624015E8|            14.056|
|              Brazil|2.5143187416E9|14.467666666666666|
|St. Vincent and t...|  2.16156007E7|            15.011|
+--------------------+--------------+------------------+
only showing top 10 rows



##### 9. ¿Hay alguna relación entre los nacimientos y el indicador DT.AMT.DLXF.CD? ¿Cómo podrías demostrarlo?

En primer lugar, se empieza obteniendo los totales de la tasa de natalidad y del indicador DT.AMT.DLXF.CD para poder compararlos. Estos se ordenan de mayor a menor, para obtener información acerca de los países con mayor deuda y los países con más nacimientos y comprobar si hay alguna relación.

In [48]:
suma_indicador = i_deuda.groupBy("country_name").agg(f.sum('dt_amt_dlxf_cd').alias('suma_indicador'))

suma_indicador.show()

+--------------------+----------------+
|        country_name|  suma_indicador|
+--------------------+----------------+
|          South Asia|4.87562958982E10|
|                Chad|    1.78106958E8|
|            Paraguay|  1.5285085736E9|
|    Congo, Dem. Rep.|   2.598365414E8|
|             Senegal|    4.44722556E8|
|          Cabo Verde|    6.77284459E7|
|Least developed c...|2.51970292994E10|
|      Macedonia, FYR|   8.758465385E8|
|              Guyana|   1.082646655E8|
|             Eritrea|    3.11100778E7|
|         Philippines|  6.4791709177E9|
|            Djibouti|    8.31851883E7|
|               Tonga|    1.03699106E7|
|                Fiji|    4.65325085E7|
|              Turkey|5.15550310058E10|
|              Malawi|    5.74223731E7|
|             Comoros|       4792269.1|
|         Afghanistan|   1.008471819E8|
|            Cambodia|   7.759559813E8|
|              Jordan|  1.9903434792E9|
+--------------------+----------------+
only showing top 20 rows



Estos son los 10 países que aparecen con un valor más algo en el indicador.

In [188]:
suma_indicador = suma_indicador.orderBy("suma_indicador", ascending = False)
suma_indicador.show(10)

+--------------------+----------------+
|        country_name|  suma_indicador|
+--------------------+----------------+
|               China|9.62186208357E10|
|              Brazil|9.00418403041E10|
|  Russian Federation|6.65897618335E10|
|              Turkey|5.15550310058E10|
|          South Asia|4.87562958982E10|
|               India|3.19235070008E10|
|           Indonesia|3.09161126538E10|
|          Kazakhstan|2.74820936864E10|
|              Mexico| 2.5218503927E10|
|Least developed c...|2.51970292994E10|
+--------------------+----------------+
only showing top 10 rows



In [50]:
suma_nacimientos = i_desarrollo.groupBy("country_name").agg(f.sum('sp_dyn_cbrt_in').alias("suma_nacimientos"))
suma_nacimientos.show()

+-------------------+------------------+
|       country_name|  suma_nacimientos|
+-------------------+------------------+
|        Philippines|            22.935|
|              Tonga|            76.087|
|           Malaysia|            51.052|
|          Singapore|              28.9|
|             Turkey|            50.152|
|             Malawi|           142.224|
|             Jordan|             48.61|
|              Palau|              39.4|
|          Sri Lanka|            33.458|
|         San Marino|              16.1|
|            Albania|            36.536|
|               Peru|            55.694|
|          St. Lucia|            12.502|
|      United States|37.099999999999994|
|            Belarus|              37.4|
|St. Kitts and Nevis|              12.6|
|              Spain|              27.0|
|           Barbados|            21.722|
| Russian Federation|              39.5|
|            Ireland|              41.7|
+-------------------+------------------+
only showing top

Aquí se pueden observar los 10 países con mayor tasa de natalidad.

In [51]:
suma_nacimientos = suma_nacimientos.orderBy("suma_nacimientos", ascending = False)
suma_nacimientos.show(10)

+--------------------+----------------+
|        country_name|suma_nacimientos|
+--------------------+----------------+
|              Malawi|         142.224|
|     Solomon Islands|         100.577|
|     Kyrgyz Republic|            81.1|
|            Botswana|          79.253|
|           Guatemala|          76.886|
|              Zambia|          76.595|
|               Tonga|          76.087|
|Micronesia, Fed. ...|          69.647|
|          Kazakhstan|           68.33|
|           Indonesia|          57.492|
+--------------------+----------------+
only showing top 10 rows



Una vez que se tiene la información anteriormente comentada, se juntan ambas columnas en una sola tabla para poder comparar mejor. No obstante, como se ha visto anteriormente, es imposible formar una tabla con ambos, tasa e indicador, de forma ordenada, de mayor a menor, ya que estos valores no coinciden. Esto muestra como existe poca relación entre ambos.

In [52]:
union3 = suma_nacimientos.join(suma_indicador, on="country_name", how = "inner")

In [53]:
union3.show(10)

+----------------+----------------+----------------+
|    country_name|suma_nacimientos|  suma_indicador|
+----------------+----------------+----------------+
|          Malawi|         142.224|    5.74223731E7|
| Solomon Islands|         100.577|    3.07497039E7|
| Kyrgyz Republic|            81.1|   8.164190451E8|
|        Botswana|          79.253|    1.37995908E8|
|       Guatemala|          76.886|  2.9119819878E9|
|          Zambia|          76.595|  1.2176434719E9|
|           Tonga|          76.087|    1.03699106E7|
|      Kazakhstan|           68.33|2.74820936864E10|
|       Indonesia|          57.492|3.09161126538E10|
|Papua New Guinea|          56.337|  2.8421938425E9|
+----------------+----------------+----------------+
only showing top 10 rows



No obstante, pese a que los datos muestran que hay poca relación, sería conveniente estimar la correlación entre ambas columnas, con el fin de averiguar si existe correlación entre indicador y tasa de nacimientos. 

In [54]:
# Para hacer la correlación es necesario pasarlo previamente a Pandas
to_pandas = union3.toPandas()


In [170]:
correlacion = union3.stat.corr("suma_indicador","suma_nacimientos")
correlacion

-0.10390308499004132

El resultado obtenido ha sido de correlacción negativa, -0.104, por lo que se podría concluir que no existe relación alguna.